In [28]:
import sys
if "/Users/raymond/code/FinalProject563" not in sys.path:
    sys.path.append("/Users/raymond/code/FinalProject563")
import torch
import pickle
import glob
import matplotlib.pyplot as plt
import random

import connect_four
import mcts
import connect_net
import connect_plot

In [4]:
files = glob.glob("/Users/raymond/code/FinalProject563/data/trial1/iteration1/examples-1-*.pickle")
files

['/Users/raymond/code/FinalProject563/data/trial1/iteration1/examples-1-0-64-128.pickle',
 '/Users/raymond/code/FinalProject563/data/trial1/iteration1/examples-1-3-64-128.pickle',
 '/Users/raymond/code/FinalProject563/data/trial1/iteration1/examples-1-2-64-128.pickle',
 '/Users/raymond/code/FinalProject563/data/trial1/iteration1/examples-1-1-64-128.pickle']

In [22]:
nnet = connect_net.load_model("/Users/raymond/code/FinalProject563/data/trial1/iteration1/model-1.model")

ConnectNet(
  (layer1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(1, 1))
    (1): ReLU()
  )
  (fc): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 4), stride=(1, 1))
    (1): ReLU()
    (2): Flatten(start_dim=1, end_dim=-1)
    (3): Linear(in_features=64, out_features=64, bias=True)
    (4): ReLU()
    (5): Linear(in_features=64, out_features=8, bias=True)
  )
)


In [5]:
with open(files[0], "rb") as f:
    examples = pickle.load(f)

In [24]:
for i, item in enumerate(list(reversed(examples))[1:20]):
    print(item)
    break

[tensor([[-0., -0., -0., -0., -0., -0., -0.],
        [-0., -0., -0., -0., -0., -0., -0.],
        [-0., -0., -0., -0., -0., -0., -0.],
        [-0., -0., -0., -0., -0., -0., -0.],
        [-0., -0., -0., -0., -0., -0., -0.],
        [-0., -0., -1., -0., -0., -0., -0.]]), tensor([0.1458, 0.1458, 0.1319, 0.1250, 0.1181, 0.1806, 0.1528]), 5, -1]


In [38]:
total_num = len(examples)
train_num = int(total_num * 0.8)
val_num = int(total_num * 0.1)
test_num = total_num - train_num - val_num
print(total_num, train_num, val_num, test_num)


my_vals = list(range(total_num))
random.shuffle(my_vals)

1263 1010 126 127


In [40]:
train_subset = [examples[i] for i in my_vals[:100]]
val_subset = [examples[i] for i in my_vals[:100]]
test_subset = [examples[i] for i in my_vals[:100]]

In [153]:
optimizer = torch.optim.Adam(nnet.parameters(), lr=0.001)
# mcts.train_nnet(nnet=nnet, examples=examples[:100], optimizer=optimizer)

In [ ]:
def loss_fn(annet, samples):
    loss_tmp = torch.zeros(1)
    for e in samples:
        v, proba = annet(e[0].view(1, 1, 6, 7))
        loss_tmp += torch.square(v - e[3]) - torch.dot(e[1], torch.log(proba.squeeze()))
    return loss_tmp

# optimizer.zero_grad()
# loss.backward()
# optimizer.step()

In [155]:
begin_val_loss = loss_fn(nnet, val_subset).item()
print(begin_val_loss)

loss = loss_fn(nnet, train_subset)
optimizer.zero_grad()
loss.backward()
optimizer.step()

end_val_loss = loss_fn(nnet, val_subset).item()
print(end_val_loss)
print(end_val_loss < begin_val_loss)

182.52589416503906
186.04083251953125
False
